In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
import os
import glob
warnings.filterwarnings('ignore')
%matplotlib inline

### Analytics - Part1 (Cohort, LTV, Retention)
- Cohort-wise # of Clients
- Cohort-wise Revenue
- Cohort-wise LTV
- Cohort-wise Retention rate
*******************************************************************************************************************************************************************
### Cohort Analysis
- A cohort is a group of customers who share something in common such as their sign-up date, first purchase date, acquisition channel, and etc. Cohort analysis is one of the analysis methods by which customers or users in the same cohort are tracked over time helping us understand the overall transactional patterns, customers' behaviors, and monitor key indicators like retention rate and revenue.

### Load data

In [2]:
path = './datasets/'
filenames = glob.glob(path + "*.csv")
file_list = []
for file in sorted(filenames):
    file_list.append(pd.read_csv(file))

customers = file_list[0]
geolocation = file_list[1]
items = file_list[2]
payments = file_list[3]
reviews = file_list[4]
orders = file_list[5]
products = file_list[6]
sellers = file_list[7]
categories = file_list[8]

### Cohort-wise # of Clients

In [18]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [19]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64